## Développement du "Modèle sur mesure avancé" avec des essais sur au moins deux word embeddings différents et en gardant celui qui permet d’obtenir les meilleures performances. Nous utiliserons Glove et FastText.
-  Nous allons construire et entraîner un réseau de neurones pour chaque embedding, évaluer les performances et enregistrer les expérimentations avec MLFlow. 
-  C'est la colonne clean_text_embeddings qui sera utilisée pour les embeddings.

### Charger le fichier CSV nettoyé

In [21]:
# Verification version de tensorflow
import tensorflow as tf
print(tf.__version__)

2.17.0


In [22]:
import os
import pandas as pd

# Chemin relatif pour charger les données nettoyées
file_path = os.path.join("..", "data", "cleaned_data_with_text_for_models.csv")

# Chargement du DataFrame nettoyé
data = pd.read_csv(file_path, index_col=0)

# Vérification du contenu du DataFrame

In [23]:
# Réinitialiser l'index pour récupérer toutes les colonnes, y compris 'text' si elle est utilisée comme index
data = data.reset_index()

# Vérifier les colonnes disponibles après réinitialisation de l'index
print(data.columns)


Index(['text', 'clean_text_tfidf', 'clean_text_embeddings', 'clean_text_bert',
       'target'],
      dtype='object')


In [24]:
# Dataframe info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 5 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   text                   1600000 non-null  object
 1   clean_text_tfidf       1592857 non-null  object
 2   clean_text_embeddings  1596608 non-null  object
 3   clean_text_bert        1600000 non-null  object
 4   target                 1600000 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 61.0+ MB


In [25]:
# Valeurs manquantes
print(data.isnull().sum())

text                        0
clean_text_tfidf         7143
clean_text_embeddings    3392
clean_text_bert             0
target                      0
dtype: int64


In [26]:
# Filtrer les lignes avec des valeurs manquantes dans la colonne 'clean_text_embeddings'
missing_embeddings = data[data['clean_text_embeddings'].isnull()]

# Afficher deux exemples pour analyser les raisons des valeurs manquantes
print(missing_embeddings[['text', 'clean_text_tfidf', 'clean_text_embeddings']].head(2))

             text clean_text_tfidf clean_text_embeddings
208    @mandayyy               NaN                   NaN
249  @mandayyy                 NaN                   NaN


In [27]:
# Supprimer les lignes avec des valeurs manquantes dans la colonne 'clean_text_embeddings'
data = data.dropna(subset=['clean_text_embeddings'])


In [28]:
# Vérifier les valeurs manquantes après suppression
print(data.isnull().sum())

text                        0
clean_text_tfidf         3751
clean_text_embeddings       0
clean_text_bert             0
target                      0
dtype: int64


### Étape 1 : Charger les embeddings GloVe et FastText
- Les liens pour télécharger les embeddings GloVe et FastText sont les suivants :
- https://nlp.stanford.edu/projects/glove/
- https://fasttext.cc/docs/en/english-vectors.html


In [29]:
import os
from gensim.models import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors

# Chemin relatif pour charger les fichiers d'embeddings
glove_path = os.path.join("..", "data", "glove.twitter.27B.100d.txt")
fasttext_path = os.path.join("..", "data", "crawl-300d-2M-subword.bin")

# Charger les embeddings GloVe
glove_model = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)

# Charger les embeddings FastText (format Facebook binaire)
fasttext_model = load_facebook_vectors(fasttext_path)


### Étape 2 : Créer une fonction pour générer une matrice d'embeddings
- Créer une matrice d'embeddings pour chaque modèle, basée sur le vocabulaire des données textuelles.

In [30]:
import numpy as np

# Fonction pour créer la matrice d'embeddings
def create_embedding_matrix(embedding_model, vocab, embedding_dim):
    embedding_matrix = np.zeros((len(vocab) + 1, embedding_dim))
    for word, i in vocab.items():
        if word in embedding_model:
            embedding_matrix[i] = embedding_model[word]
        else:
            # Si le mot n'est pas trouvé dans les embeddings, laisser un vecteur de zéros
            embedding_matrix[i] = np.zeros(embedding_dim)
    return embedding_matrix


### Étape 3 : Préparation des Données pour la vectorisation et la division en ensemble d'entrainement/test

1. **Sous-échantillonnage des Données**
2. **Définition des Paramètres de Tokenisation**
3. **Application de `TextVectorization`**
4. **Conversion des Textes en Séquences Numériques**
5. **Préparation des Labels**
6. **Division des Données en Ensemble d’Entraînement et de Test**

In [31]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

# # Sous-échantillonnage des données à 0.1 % pour tester rapidement le pipeline
data_sample = data.sample(frac=0.001, random_state=42)

# Tokenisation des textes nettoyés pour les word embeddings
num_words = 10000
max_sequence_length = 100

# Utiliser TextVectorization pour transformer les textes
tv_layer = tf.keras.layers.TextVectorization(
    max_tokens=num_words,
    output_mode='int',
    output_sequence_length=max_sequence_length
)

# Adapter TextVectorization sur les textes
tv_layer.adapt(data['clean_text_embeddings'])

# Convertir les textes en séquences
X = tv_layer(data['clean_text_embeddings'])

# Convertir le Tensor en tableau NumPy
X = X.numpy()

# Préparer les labels
y = data['target'].values  # Assurez-vous que y est également un tableau NumPy

# Division des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Etape 5. Créer et entraîner les modèles avec GloVe et FastText
- Nous allons maintenant construire et entraîner les modèles avec les embeddings GloVe et FastText.

- a) Construction du Modèle de Réseau de Neurones avec Embeddings Pré-entraînés :

In [33]:
import tensorflow as tf

# Utilisation de tf.keras pour construire le modèle
def build_model(embedding_matrix, input_length):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(input_dim=embedding_matrix.shape[0],
                                        output_dim=embedding_matrix.shape[1],
                                        weights=[embedding_matrix],
                                        input_length=input_length,
                                        trainable=False))  # Ne pas entraîner les embeddings
    model.add(tf.keras.layers.LSTM(128, return_sequences=False))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # Classification binaire
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


- b) Entraînement du modèle avec GloVe :

In [ ]:
import numpy as np
import tensorflow as tf

# Créer la matrice d'embeddings pour GloVe
def create_embedding_matrix(glove_model, tv_layer, embedding_dim):
    # Récupérer l'index des mots de TextVectorization
    word_index = {word: index for index, word in enumerate(tv_layer.get_vocabulary())}
    
    # Initialiser la matrice d'embeddings
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in glove_model:
            embedding_vector = glove_model[word]
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Définir le nombre de mots et la longueur maximale des séquences
num_words = 10000
max_sequence_length = 100

# Initialiser la couche TextVectorization
tv_layer = tf.keras.layers.TextVectorization(
    max_tokens=num_words,
    output_mode='int',
    output_sequence_length=max_sequence_length
)

# Adapter TextVectorization sur les textes nettoyés
tv_layer.adapt(data['clean_text_embeddings'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Définir la dimension des embeddings pour GloVe
embedding_dim_glove = 100  # Taille des vecteurs d'embeddings pour GloVe, ajustez selon le modèle GloVe utilisé

# Créer la matrice d'embeddings en utilisant TextVectorization (tv_layer)
embedding_matrix_glove = create_embedding_matrix(glove_model, tv_layer, embedding_dim_glove)

# Construire le modèle avec GloVe
model_glove = build_model(embedding_matrix_glove, input_length=max_sequence_length)

# Définir un callback pour sauvegarder le meilleur modèle en fonction de la perte de validation
checkpoint_cb = ModelCheckpoint('best_model_glove.h5', save_best_only=True, monitor='val_loss', mode='min')

# Définir un callback pour arrêter l'entraînement si la perte de validation ne s'améliore pas après 3 époques
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Créer une liste de callbacks pour le modèle
callbacks = [checkpoint_cb, early_stopping_cb]

# Entraîner le modèle avec les callbacks
history_glove = model_glove.fit(
    X_train, y_train, 
    epochs=20,  # Augmenter le nombre d'époques pour permettre un entraînement suffisant avec early stopping
    batch_size=32, 
    validation_data=(X_test, y_test), 
    callbacks=callbacks
)

# Récupérer la meilleure époque atteinte
best_epoch_glove = early_stopping_cb.stopped_epoch - early_stopping_cb.patience + 1 if early_stopping_cb.stopped_epoch else 20
print(f"La meilleure époque pour GloVe est : {best_epoch_glove}")

# Enregistrer la meilleure époque et le batch_size pour GloVe
best_batch_size_glove = 32  # Celui que vous avez utilisé pour l'entraînement
best_epochs_glove = best_epoch_glove

- c) Entraînement du modèle avec FastText :

In [ ]:
import numpy as np
import tensorflow as tf

# Créer la matrice d'embeddings pour FastText
def create_embedding_matrix(fasttext_model, tv_layer, embedding_dim):
    # Récupérer l'index des mots de TextVectorization
    word_index = {word: index for index, word in enumerate(tv_layer.get_vocabulary())}
    
    # Initialiser la matrice d'embeddings
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in fasttext_model:
            embedding_vector = fasttext_model[word]
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Définir le nombre de mots et la longueur maximale des séquences
num_words = 10000
max_sequence_length = 100

# Initialiser la couche TextVectorization
tv_layer = tf.keras.layers.TextVectorization(
    max_tokens=num_words,
    output_mode='int',
    output_sequence_length=max_sequence_length
)

# Adapter TextVectorization sur les textes nettoyés
tv_layer.adapt(data['clean_text_embeddings'])


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Définir la dimension des embeddings pour FastText
embedding_dim_fasttext = 300  # Assurez-vous que cela correspond à la dimension de vos vecteurs FastText

# Créer la matrice d'embeddings en utilisant TextVectorization (tv_layer)
embedding_matrix_fasttext = create_embedding_matrix(fasttext_model, tv_layer, embedding_dim_fasttext)

# Construire le modèle avec FastText
model_fasttext = build_model(embedding_matrix_fasttext, input_length=max_sequence_length)

# Définir un callback pour sauvegarder le meilleur modèle en fonction de la perte de validation
checkpoint_cb = ModelCheckpoint('best_model_fasttext.h5', save_best_only=True, monitor='val_loss', mode='min')

# Définir un callback pour arrêter l'entraînement si la perte de validation ne s'améliore pas après 3 époques
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Créer une liste de callbacks pour le modèle
callbacks = [checkpoint_cb, early_stopping_cb]

# Entraîner le modèle avec les callbacks
history_fasttext = model_fasttext.fit(
    X_train, y_train,
    epochs=20,  # Augmenter le nombre d'époques pour permettre un entraînement suffisant avec early stopping
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=callbacks
)


# Récupérer la meilleure époque atteinte
best_epoch_fasttext = early_stopping_cb.stopped_epoch - early_stopping_cb.patience + 1 if early_stopping_cb.stopped_epoch else 20
print(f"La meilleure époque pour FastText est : {best_epoch_fasttext}")

# Enregistrer les paramètres optimaux
best_batch_size_fasttext = 32  # Celui que vous avez utilisé
best_epochs_fasttext = best_epoch_fasttext



### Définition de la fonction evaluate_model pour évaluer les performances des modèles :

#### Vérifier l'initialisation de MLFlow

In [ ]:
import mlflow
import mlflow.keras

# Définir l'expérience MLFlow
mlflow.set_experiment("text_classification_experiment")

#### Fonction pour évaluer les performances du modèle :

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import mlflow
import mlflow.keras

def evaluate_model(model, X_test, y_test, model_name):
    # Prédictions du modèle
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    y_pred_proba = model.predict(X_test)
    
    # Calcul des métriques
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Enregistrement des métriques dans MLFlow
    with mlflow.start_run(run_name=model_name):
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("roc_auc", roc_auc)

        # Tracer la courbe ROC
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        plt.figure()
        plt.plot(fpr, tpr, marker='.')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve for {model_name}')
        roc_curve_path = f"roc_curve_{model_name}.png"
        plt.savefig(roc_curve_path)
        mlflow.log_artifact(roc_curve_path)
        plt.close()

    print(f"Model evaluation for {model_name} completed.")


### 6. Enregistrer les expérimentations avec MLFlow

- a) Enregistrer les résultats pour GloVe :

In [ ]:
from pathlib import Path
import mlflow
import mlflow.keras

# Chemin absolu pour le dossier "mlruns"
mlruns_path = Path("../mlruns").resolve()

# Vérifier que le dossier "mlruns" existe, sinon le créer
if not mlruns_path.exists():
    mlruns_path.mkdir(parents=True)

# Vérifier que le sous-dossier ".trash" existe, sinon le créer
trash_folder = mlruns_path / ".trash"
if not trash_folder.exists():
    trash_folder.mkdir(parents=True)

# Configuration du chemin pour stocker les artefacts de MLFlow
mlflow.set_tracking_uri(mlruns_path.as_uri())

# Définir le nom de l'expérience
experiment_name = "GloVe_Embedding_Experiment"
mlflow.set_experiment(experiment_name)

# Démarrer une nouvelle session MLFlow pour GloVe
with mlflow.start_run(run_name="GloVe_Model_Evaluation", nested=True) as run:
    # Enregistrer les paramètres du modèle
    mlflow.log_param("embedding", "GloVe")
    mlflow.log_param("embedding_dim", embedding_dim_glove)
    mlflow.log_param("batch_size", best_batch_size_glove)
    mlflow.log_param("epochs", best_epochs_glove)

    # Évaluer le modèle et enregistrer les métriques et la courbe ROC
    evaluate_model(model_glove, X_test, y_test, "GloVe")

    # Enregistrer le modèle Keras avec MLFlow
    mlflow.keras.log_model(model_glove, "glove_model")

    # Ajouter des informations sur le run
    mlflow.log_artifact(mlruns_path)
    mlflow.log_param("model_path", "glove_model")
    mlflow.log_param("experiment_id", run.info.experiment_id)
    mlflow.log_param("run_id", run.info.run_id)

print("Modèle GloVe évalué et enregistré dans MLFlow.")


- b) Enregistrer les résultats pour FastText :

In [ ]:
from pathlib import Path
import mlflow
import mlflow.keras

# Chemin absolu pour le dossier "mlruns"
mlruns_path = Path("../mlruns").resolve()

# Vérifier que le dossier "mlruns" existe, sinon le créer
if not mlruns_path.exists():
    mlruns_path.mkdir(parents=True)

# Vérifier que le sous-dossier ".trash" existe, sinon le créer
trash_folder = mlruns_path / ".trash"
if not trash_folder.exists():
    trash_folder.mkdir(parents=True)

# Configuration du chemin pour stocker les artefacts de MLFlow
mlflow.set_tracking_uri(mlruns_path.as_uri())

# Définir le nom de l'expérience
experiment_name = "FastText_Embedding_Experiment"
mlflow.set_experiment(experiment_name)

# Démarrer une nouvelle session MLFlow pour FastText
with mlflow.start_run(run_name="FastText_Model_Evaluation", nested=True) as run:
    # Enregistrer les paramètres du modèle
    mlflow.log_param("embedding", "FastText")
    mlflow.log_param("embedding_dim", embedding_dim_fasttext)
    mlflow.log_param("batch_size", best_batch_size_fasttext)
    mlflow.log_param("epochs", best_epochs_fasttext)

    # Évaluer le modèle et enregistrer les métriques et la courbe ROC
    evaluate_model(model_fasttext, X_test, y_test, "FastText")

    # Enregistrer le modèle Keras avec MLFlow
    mlflow.keras.log_model(model_fasttext, "fasttext_model")

    # Ajouter des informations sur le run
    mlflow.log_artifact(mlruns_path)
    mlflow.log_param("model_path", "fasttext_model")
    mlflow.log_param("experiment_id", run.info.experiment_id)
    mlflow.log_param("run_id", run.info.run_id)

print("Modèle FastText évalué et enregistré dans MLFlow.")


### 7. Conclusion
- Ce code utilise le fichier cleaned_data_with_text_for_models.csv et applique les word embeddings GloVe et FastText. Nous avons modifié le code pour utiliser les textes déjà nettoyés dans la colonne clean_text_embeddings. Après avoir entraîné les modèles, nous enregistrons les résultats et les modèles dans MLFlow afin de comparer les performances des deux embeddings et de sélectionner le meilleur.